In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import html5lib
%matplotlib inline

In [2]:
def theurl(name):
    answer={"Obama B":"http://www.presidency.ucsb.edu/data/popularity.php?pres=44&sort=time&direct=DESC&Submit=DISPLAY",
            "Bush G" : "http://www.presidency.ucsb.edu/data/popularity.php?pres=43&sort=time&direct=DESC&Submit=DISPLAY",
            "Clinton B": "http://www.presidency.ucsb.edu/data/popularity.php?pres=42&sort=time&direct=DESC&Submit=DISPLAY"
           }
    return answer[name]

In [3]:
def make_soup(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.content,"lxml")
    return soup

In [4]:
this=make_soup(theurl("Obama B"))

In [5]:
tables = this.findChildren('table')

In [6]:
table = tables[10]

In [46]:
data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]

In [47]:
headers = data[4]

In [48]:
body =data[6:]

In [49]:
cols   =  zip(*body)

In [50]:
tbl_d  = {name:col for name, col in zip(headers,cols)}

In [56]:
pd.DataFrame(tbl_d)

,,Approving,Disapproving,End Date,President,Start Date,unsure/no data
0,,48,48,04/13/2016,Barack Obama,04/11/2016,4
1,,47,48,04/12/2016,,04/10/2016,5
2,,48,48,04/11/2016,,04/09/2016,4
3,,49,46,04/10/2016,,04/08/2016,5
4,,51,45,04/09/2016,,04/07/2016,4
5,,52,44,04/08/2016,,04/06/2016,4
6,,52,44,04/07/2016,,04/05/2016,4
7,,52,44,04/06/2016,,04/04/2016,4
8,,51,44,04/05/2016,,04/03/2016,5
9,,53,43,04/04/2016,,04/02/2016,4


In [69]:
def presidentialDataFramer(soup):
    this=make_soup(theurl(soup))
    
    # find all tables then specifiy which table
    tables = this.findChildren('table')
    table = tables[10]
    
    # extract the data
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    # create the headers
    headers = data[4]
    
    # find the body of the data and zip it together
    body =data[6:]
    cols   =  zip(*body)
    tbl_d  = {name:col for name, col in zip(headers,cols)}
    
    # create a dataframe from the html and trim the first column off
    df = pd.DataFrame(tbl_d)
    trimmed = df.drop(df.columns[[0]], axis=1)
    return trimmed

In [70]:
clint=presidentialDataFramer("Clinton B")

In [68]:
clint

,Approving,Disapproving,End Date,President,Start Date,unsure/no data
0,66,29,01/14/2001,William J. Clinton,01/10/2001,5
1,63,31,01/07/2001,,01/05/2001,4
2,71,26,12/17/2000,,12/15/2000,1
3,59,35,12/04/2000,,12/02/2000,5
4,63,32,11/15/2000,,11/13/2000,4
5,56,38,10/26/2000,,10/25/2000,5
6,58,36,10/09/2000,,10/06/2000,4
7,60,34,09/13/2000,,09/11/2000,5
8,62,35,09/03/2000,,08/29/2000,2
9,61,35,08/19/2000,,08/18/2000,3
